In [1]:
import os, re, csv
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm
import gensim
import multiprocessing
from kiwipiepy import Kiwi
from soyspacing.countbase import CountSpace

In [2]:
#corpus 읽어서 태그달기(태그는 임의의 태그, 입력된 순서대로 달림)
def read_corpus(noun_content):
    for i, content in tqdm(enumerate(noun_content)):
        yield gensim.models.doc2vec.TaggedDocument(content, [i])

In [4]:
df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\Q&A.csv", encoding='cp949')
lem = df["LEMMA"].tolist()
print(lem[:10])
lem_cont = []
for k in lem:
    lem_cont.append(str(k).split())
print(lem_cont[:10])

['이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시', '재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사항 이전 성적 연결 현황 동일 교과 자동연결되었습 나오 학사 공지 나오 대로 재수강 이전 성적 선택 재수강 연결 필요 아무것 감사', '수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제', '처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울 방학 2019 부전공 경영 선택 2020년 학기 이중전공 변경 기간 경영 신청 한번 종합 정보 시스템 들어가 사진 내요 신경 변경 기간 경영 신청', '이중전공 바꾸 바꾸 학점 소멸', '등록 휴학 자퇴 경우 이번 학기 등록금 환불 가능', '졸업 유예 기간 동안 학점 취득 가능', '인간 사회 학점 짜리 과목 씨쁠 이거 언어 문학 학점 짜리 페논 과목 재수강 연결 가능', '2019 학기 학기 마치 사정 학점 상태 휴학 전공 이중전공 학점 채운 상황 취득 학점 132 학점 학점 모자라 추가 학기 상황 경우 2020 여름 계절 학기 학점 수강 수료 가능 답변 부탁 드리', '교양 영어 수강 학생 온라인 과제 코드']
[['이중전공', '부전공', '차이', '대하', '이중전공', '학위', '졸업장', '표시', '부전공', '졸업장', '표시'], ['재수강', '연결', '종합', '정보', '시스템', '수업', '수강', '관리', '수강', '과목', '선택', '메뉴', '들어가', '안내', '사항', '이전', '성적', '연결', '현황', '동일', '교과', '자동연결되었습', '나오', '학사', '공지', '나오', '대로', '재수강', '이전', '성적', '선택', '재수강', '연결', '필요', '아무것', '감사'], ['수고', '다르', '아니', '제가', '평점', '학기', '수업', '평점', '올리', '졸업', '학점', '134', '학점', '

In [4]:
corpus = list(read_corpus(lem_cont))
print(corpus[:10])

cores = multiprocessing.cpu_count()

#doc2vec parameters
vector_size = 1000
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = cores #number of parallel processes

#모델 학습, doc2vec
model = gensim.models.doc2vec.Doc2Vec(
    min_count=word_min_count, size=vector_size, alpha=0.025, min_alpha=0.025, seed=1234, workers=worker_count
    )

#모델 생성
model.build_vocab(corpus) #모델 훈련 위한 vocab 생성

# Train document vectors!
for epoch in range(10):
	model.train(corpus, total_examples=model.corpus_count, epochs=train_epoch)
	model.alpha -= 0.002 # decrease the learning rate
	model.min_alpha = model.alpha # fix the learning rate, no decay

# To save
model.save(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\models\doc2vec.model")
model.save_word2vec_format(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\models\doc2vec.model.word2vec_format", binary=False)


7015it [00:00, 701350.18it/s]
[TaggedDocument(words=['이중전공', '부전공', '차이', '대하', '이중전공', '학위', '졸업장', '표시', '부전공', '졸업장', '표시'], tags=[0]), TaggedDocument(words=['재수강', '연결', '종합', '정보', '시스템', '수업', '수강', '관리', '수강', '과목', '선택', '메뉴', '들어가', '안내', '사항', '이전', '성적', '연결', '현황', '동일', '교과', '자동연결되었습', '나오', '학사', '공지', '나오', '대로', '재수강', '이전', '성적', '선택', '재수강', '연결', '필요', '아무것', '감사'], tags=[1]), TaggedDocument(words=['수고', '다르', '아니', '제가', '평점', '학기', '수업', '평점', '올리', '졸업', '학점', '134', '학점', '초과', '문제'], tags=[2]), TaggedDocument(words=['처음', '이중전공', '신청', '이중전공', '경영', '변경', '시도', '자체', '시험', '떨어지', '변경', '실패', '지나', '겨울', '방학', '2019', '부전공', '경영', '선택', '2020년', '학기', '이중전공', '변경', '기간', '경영', '신청', '한번', '종합', '정보', '시스템', '들어가', '사진', '내요', '신경', '변경', '기간', '경영', '신청'], tags=[3]), TaggedDocument(words=['이중전공', '바꾸', '바꾸', '학점', '소멸'], tags=[4]), TaggedDocument(words=['등록', '휴학', '자퇴', '경우', '이번', '학기', '등록금', '환불', '가능'], tags=[5]), TaggedDocument(words=['졸업', '유예', '기간', '동안

In [5]:
def preprocess(question, space_model_path, stop_word_path):
    model = CountSpace()
    model.load_model(space_model_path, json_format=False)

    # 띄어쓰기 시작
    soy_corrected_question = []
    corrected = model.correct(question)
    soy_corrected_question.append(corrected)

    spaced_quest = []
    for quest, tag in soy_corrected_question:
        if type(quest) == str:
            spaced_quest.append(quest)
        else:
            continue


    # 정규표현식으로 정규화
    norm_content = []
    for text in spaced_quest:
        punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
        norm_text = re.sub(punct, "", text)
        norm_text = re.sub(r'(안녕)+(하세요)?', '', norm_text)
        norm_text = re.sub(r'(조교)[님]?', "", norm_text)
        norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', norm_text) # ㅠ하나 쓰는 놈들꺼 제거...
        norm_text = re.sub( r"\s{2,}"," ", norm_text) 
        norm_text = re.sub( r"\A\s+",'', norm_text)
        norm_text = re.sub(r"([가-힣]+)[여용염]",r"\1요",norm_text) # 용용체 제거 슈바끄...
        norm_text = re.sub(r"\b이중[^전공]*?\b","이중전공",norm_text) # 이중으로만 적혀있는거 변형
        norm_text = re.sub(r"\b부전[^공]*?\b","부전공",norm_text) # 부전으로만 적혀있는거 변형

        # 줄인말제거
        norm_text = re.sub(r"\b전심[가-힣]*\b", "전공심화", norm_text)
        norm_text = re.sub(r"\b융전[가-힣]*\b", "융합전공", norm_text)
        norm_text = re.sub(r'\b연계전공\b', '융합전공', norm_text) # 단어 통일화
        
        norm_content.append(norm_text)

        # 형태소 분석

        kiwi = Kiwi()

        #대분류 사전 추가
        kiwi.add_user_word('이중전공', 'DM', 20.0)
        kiwi.add_user_word('융합전공', 'UM', 20.0)
        kiwi.add_user_word('전공심화', 'MD', 20.0)
        kiwi.add_user_word('부전공', 'SM', 20.0)
        kiwi.add_user_word('연계전공', 'UM', 20.0)

        kiwi.prepare()

        ## 형태소 분석
        sent = ''
        lem_cont = []
        for quest in norm_content:
            ## 형태소만 떼어내기
            temp_tagging = [x[0] for x in kiwi.analyze(quest, top_n=1)]
            inner_temp = ["{}/{}".format(word, tag) for word, tag, score1, score2 in temp_tagging[0]]
            lem_cont.append(inner_temp)

        ## 불용어 처리 ##

        stop_file = open(stop_word_path,'r', encoding = 'utf8')
        lines = stop_file.readlines()
        stop_file.close()
        stop_words = lines[0].split()

        listed_cont = [[comb.split('/') for comb in sent] for sent in lem_cont]

        main_texts = []
        non_mean = []
        for sent in listed_cont:
            for word in sent:
                # 한 글자 제거
                if len(word[0]) > 1:
                    # 조사 등 불필요한 거 제거
                    if 'N' in word[-1]:
                        main_texts.append(word[0])
                    elif '@' in word[-1]:
                        main_texts.append(word[0])
                    elif 'V' in word[-1]:
                        main_texts.append(word[0])
                    else:
                        non_mean.append(word[0])
                else:
                    non_mean.append(word[0])


    return main_texts

In [19]:
content = df["QUESTION"].tolist()
comments = df["ANSWER"].tolist()
print(comments[3000:3100])

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, "['종합정보시스템 > 학적변동신청 > 자퇴신청을 한 다음, 자퇴원서를 작성을 해서 학종지에 제출하면 됩니다']", "['1. 등록금 환불은 자퇴 시에만 가능합니다 휴학을 하면 등록휴학으로 처리됩니다 2. 지금 코로나 때문에 학사일정이 다소 바뀌었는데 학기 1/4선 계산 등은 학종지에 직접 문의하셔서 확인바랍니다', '현재 해외에 있어서 학종지에 직접 문의를 하기가 힘든 상황입니다', '그런 경우엔 보통 지인이나 가족을 통해서 문의를 합니다 저도 예전엔 그랬고요..', '아 넵 알겠습니다']", "['음 지금 이게 전공인지 교양인지 또 (1),(2)로 나뉘어진 연강수업인지 아니면 학기 상관없는 동일수업인지 알 수가 없어서 답변이 제한되네요...우선 종정시에 접속하셔서 수업/수강관리 > 재수강과목선택 부분을 확인바랍니다', '전공수업에 1,2 나눠지지 않은 동일과목입니다! 자동연결될줄 알았는데 학교에서 미연결과목있다고 문자오고 재수강과목선택 들어가서 성적연결신청을 누르면 이전성적 조회결과가 없다고 뜹니다ㅠㅠ 교수님까지 똑같은 강의인데ㅠㅠ 자동연결이 되는걸까요ㅠㅠ', '그럼 자동연결이 되었을텐데 이상하네요 혹시 전공 말고 교양 등 다른 과목이 아닌지 확인바라며 만일 그것도 아니라면 학종지에 문의바랍니다', '원래 재수강 연결할때 이전성적이 뜨고 연결할수있는건가요? 전 아무리봐도 같은 과목인데 자동연결이라고 안써져있고..ㅠㅠ']", "['미네르바는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']", "['대학외국어는 교양필수과목이기 때문에 타교양으로 재수강 연결을 할 수 없습니다 (동일과목으로만 재수강 가능)']", "['조교를 거치지 않고 교수님 개인 메일로 보내는 거라면 안심하셔도 됩니다 다른 학생들에게 노출되지 않도록 교수님

In [23]:
#main
query = input('궁금하신 사항을 입력해 주세요: ')
# query = [query] #전처리에 인수 값으로 리스트 형식으로 넣어줘야 하기 때문에 문자열을 포함한 리스트 형식으로 변경

# gensim model 불러오기
model = gensim.models.Doc2Vec.load(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\doc2vec.model")

##전처리 진행, 전처리한 문장을 학습한 doc2vec모델에 적용해 vectorize
space_path = r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\soy_big_space_model"
stop_path = r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\stop_words.txt"
preprocessed = preprocess(query, space_model_path=space_path, stop_word_path=stop_path)
print(preprocessed)
vector = model.infer_vector(preprocessed) #학습된 모델로 벡터값 유추

##가장 유사한 vector를 갖는 질문들 모아모아
sims = model.docvecs.most_similar([vector], topn=len(model.docvecs))

##유사도 상위 10개 tag들
sims_tags = list(map(lambda x: x[0], sims[:10])) 

print('\n\n')
for i, tag in enumerate(sims_tags):
    print(i+1, ')', '유사도: ', sims[i][1])
    print(tag)
    print('------질문------')
    print(content[tag])
    print('\n')
    
    print('------답변------')
    comment_devided = comments[tag]
    print(comment_devided)
    # for j, comment in enumerate(comment_devided):
    #     print(j+1,')', comment)
    print('\n\n\n\n')

['서울캠', '전공']



1 ) 유사도:  0.8341413140296936
6890
------질문------
서울캠에서 네덜란드어전공을 들을 수 있나요


------답변------
아니요





2 ) 유사도:  0.7746298313140869
6891
------질문------
서울캠에서 경영학 들을 수 있나요


------답변------
아니요





3 ) 유사도:  0.74948650598526
6923
------질문------
광고pr브랜딩 전공인데 서울캠에서 들어요


------답변------
이중전공은 양 캠퍼스에서 가능하나, 부전공은 글로벌캠퍼스에서 가능합니다.





4 ) 유사도:  0.6375558376312256
3665
------질문------
이번에 계절학기를 서울캠에서 듣게 되었는데 789교시이면 서울캠은 3시부터 수업이라는데 맞나요


------답변------
['네 서울캠은 정각이 기준입니다']





5 ) 유사도:  0.6335957646369934
639
------질문------
이번에 계절학기를 서울캠에서 듣게 되었는데 789교시이면 서울캠은 3시부터 수업이라는데 맞나요


------답변------
nan





6 ) 유사도:  0.633027970790863
988
------질문------
학사경고는 징계인가요


------답변------
nan





7 ) 유사도:  0.6124325394630432
4013
------질문------
학사경고는 징계인가요


------답변------
['바아보보봉', '보통 징계라고 하는건 학칙 상 물의를 일으킨 학생을 대상으로 징계위원회의 심의를 거쳐 규정에 의거해 징계하는 것을 의미합니다 학사경고는 해당되지 않습니다']





8 ) 유사도:  0.5975210666656494
2734
------질문------
자퇴하려면 어떤 절차를 거쳐야하나요


------답변------
nan





9 ) 유사도:  0.58478975296